In [7]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI

class GeneralResponse(BaseModel):
    player_id: int = Field(description="ID of the player")
    report_summary: str = Field(name="report_summary", description="Summary of the reports that have the same player id")
    

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"
    ),
    mode=instructor.Mode.JSON
)    


In [9]:
resp = client.chat.completions.create(
    model="mistral",
    messages=[{
        "role": "user",
        "content": "Played as an attacking winger and used his speed and technique excellently. His dribbling led to several dangerous situations, although he again showed weaknesses defensively. He set up a goal with a precise cross and scored one himself after a solo run. His creativity and game vision were at a high level. At 23 years old, he still has a lot of potential to rise.",
    }],
    response_model=GeneralResponse,
)

print(resp.model_dump_json(indent=2))

{
  "player_id": 123456,
  "report_summary": "Played as an attacking winger, showcased speed and technique. Dribbling led to dangerous situations but showed weaknesses defensively. Set up a goal with a precise cross and scored one himself after a solo run. High creativity and game vision. Potential for further growth at 23 years old."
}
